# Cubic Equations of State

Simulating the compressibility factor of methane across a pressure sweep using the Peng-Robinson and Soave-Redlich-Kwong equations of state wired into a PathSim simulation.

Cubic equations of state compute the compressibility factor $Z = Pv/(RT)$ by solving a cubic polynomial at each $(T, P)$ condition. The EoS blocks take two inputs (temperature and pressure) and produce two outputs (molar volume and compressibility factor).

At low pressure $Z \to 1$ (ideal gas). At moderate pressure attractive forces cause $Z < 1$, and at high pressure excluded-volume effects push $Z > 1$.

In [ ]:
import matplotlib.pyplot as plt

from pathsim import Simulation, Connection
from pathsim.blocks import Source, Constant, Scope

from pathsim_chem.thermodynamics import PengRobinson, RedlichKwongSoave

## System Definition

Create Peng-Robinson and SRK blocks for pure methane. A `Constant` block supplies a fixed temperature while a `Source` sweeps pressure from 0.1 MPa to 30 MPa.

In [ ]:
# Critical properties of methane
Tc, Pc, omega = 190.6, 4.6e6, 0.011

# EoS blocks
pr = PengRobinson(Tc=Tc, Pc=Pc, omega=omega)
rks = RedlichKwongSoave(Tc=Tc, Pc=Pc, omega=omega)

# Fixed temperature, logarithmic pressure sweep
import numpy as np
T_const = Constant(250)  # 250 K (above Tc, supercritical)
P_src = Source(func=lambda t: 10**(4 + t * 0.035))  # 10 kPa to ~30 MPa over 100s

# Scopes: record Z from both EoS (output port 1)
scp_pr = Scope(labels=["Z_PR"])
scp_rks = Scope(labels=["Z_RKS"])

## Wiring

Both EoS blocks receive the same $(T, P)$ inputs. We record the compressibility factor (output port 1) from each.

In [ ]:
sim = Simulation(
    blocks=[T_const, P_src, pr, rks, scp_pr, scp_rks],
    connections=[
        # Temperature -> both EoS (input port 0)
        Connection(T_const, pr, rks),
        # Pressure -> both EoS (input port 1)
        Connection(P_src, pr[1], rks[1]),
        # Z output (port 1) -> scopes
        Connection(pr[1], scp_pr),
        Connection(rks[1], scp_rks),
    ],
    dt=1.0,
)

sim.run(100)

In [ ]:
time, Z_pr = scp_pr.read()
_, Z_rks = scp_rks.read()
P_vals = 10**(4 + time * 0.035) / 1e6  # MPa

fig, ax = plt.subplots(figsize=(7, 5))
ax.semilogx(P_vals, Z_pr[0], label="Peng-Robinson")
ax.semilogx(P_vals, Z_rks[0], "--", label="Soave-Redlich-Kwong")
ax.axhline(1.0, color="gray", linestyle="-.", alpha=0.5, label="Ideal gas")
ax.set_xlabel("Pressure [MPa]")
ax.set_ylabel("Compressibility Factor Z")
ax.set_title("Methane at T = 250 K")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Both EoS give very similar results. The characteristic dip below $Z = 1$ at moderate pressures reflects attractive intermolecular forces, while the rise above $Z = 1$ at high pressures is due to repulsive (excluded volume) effects.

## Mixture

The EoS blocks also support mixtures through van der Waals one-fluid mixing rules. Here we set up a methane-ethane mixture and sweep pressure.

In [ ]:
pr_mix = PengRobinson(
    Tc=[190.6, 305.3],
    Pc=[4.6e6, 4.872e6],
    omega=[0.011, 0.099],
    x=[0.7, 0.3],
)

T_const2 = Constant(300)
P_src2 = Source(func=lambda t: 10**(4 + t * 0.035))
scp_mix = Scope(labels=["Z_mixture"])

sim_mix = Simulation(
    blocks=[T_const2, P_src2, pr_mix, scp_mix],
    connections=[
        Connection(T_const2, pr_mix),
        Connection(P_src2, pr_mix[1]),
        Connection(pr_mix[1], scp_mix),
    ],
    dt=1.0,
)

sim_mix.run(100)

In [ ]:
time_m, Z_mix = scp_mix.read()
P_mix = 10**(4 + time_m * 0.035) / 1e6

fig, ax = plt.subplots(figsize=(7, 5))
ax.semilogx(P_vals, Z_pr[0], label="Pure CH₄ (250 K)")
ax.semilogx(P_mix, Z_mix[0], "--", label="70/30 CH₄-C₂H₆ (300 K)")
ax.axhline(1.0, color="gray", linestyle="-.", alpha=0.5)
ax.set_xlabel("Pressure [MPa]")
ax.set_ylabel("Compressibility Factor Z")
ax.set_title("Peng-Robinson: Pure vs Mixture")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

The mixture shows a deeper dip because ethane ($T_c = 305.3$ K) is near its critical temperature at 300 K, leading to stronger non-ideal behavior.